# Main

In [ ]:
#!pip install ray[rllib,tune]
#!pip install pettingzoo pygame pymunk
#!pip install torch

In [1]:
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-06 19:59:19,890	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-06 19:59:22,078	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import sys
from glob import glob
from os import path
from ray.rllib.policy.policy import Policy

ray.shutdown() 

# Trained to about 0 combined return
checkpoint_path = "/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
#specs = {path.basename(p) : SingleAgentRLModuleSpec(load_state_path=p) for p in pols} # Non-deterministic policy weight return (implies new)


num_agents = 2

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        #model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            #load_state_path=
            #module_specs=specs,
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)

resto_algo = resto_config.build()
resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()

2024-08-29 02:06:20,467	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.

Above, we restore an algo to its original size, and evaluate below.

In [7]:
resto_algo.evaluate()

{'env_runners': {'episode_reward_max': np.float64(104.48447046235398),
  'episode_reward_min': np.float64(-49.97758701259906),
  'episode_reward_mean': np.float64(34.31495169008534),
  'episode_len_mean': np.float64(500.0),
  'episode_media': {},
  'episodes_timesteps_total': 5000,
  'policy_reward_min': {'pursuer_0': np.float64(-27.80399612520721),
   'pursuer_1': np.float64(-74.73581968384396)},
  'policy_reward_max': {'pursuer_0': np.float64(108.50397211584374),
   'pursuer_1': np.float64(66.59594375510522)},
  'policy_reward_mean': {'pursuer_0': np.float64(32.97170784631929),
   'pursuer_1': np.float64(1.3432438437660394)},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [np.float64(44.487376281462424),
    np.float64(98.65380257932057),
    np.float64(84.50964155764572),
    np.float64(20.08502968193733),
    np.float64(104.48447046235398),
    np.float64(-38.577462399422544),
    np.float64(-49.97758701259906),
    np.float64(20.465628189378837),
    np.float64(91.2633

Next, lets try with a different number of test agents.

In [3]:
import sys
from glob import glob
from os import path
from ray.rllib.policy.policy import Policy
import numpy as np

ray.shutdown() 

# Trained to about 0 combined return
checkpoint_path = "/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
#specs = {path.basename(p) : SingleAgentRLModuleSpec(load_state_path=p) for p in pols} # Non-deterministic policy weight return (implies new)

num_trained_agents = 2
num_test_agents = 6

register_env(f"{num_test_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_test_agents)))
policies = {f"pursuer_{i}" for i in range(num_test_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_test_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        #model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            #load_state_path=
            #module_specs=specs,
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)

resto_algo = resto_config.build()
for test_id in range(num_test_agents):
    train_id = np.random.randint(num_trained_agents)
    resto_algo.get_policy(f"pursuer_{test_id}").set_weights(specs[f"pursuer_{train_id}"].get_weights())

#resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
#resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()

resto_algo.evaluate()

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-09-03 17:50:57,409	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py

{'env_runners': {'episode_reward_max': np.float64(-133.08651258384768),
  'episode_reward_min': np.float64(-429.6043485772751),
  'episode_reward_mean': np.float64(-279.01460767396213),
  'episode_len_mean': np.float64(500.0),
  'episode_media': {},
  'episodes_timesteps_total': 5000,
  'policy_reward_min': {'pursuer_0': np.float64(-70.69341337433875),
   'pursuer_1': np.float64(-83.28156152153042),
   'pursuer_2': np.float64(-89.37824978401237),
   'pursuer_3': np.float64(-64.26033845731828),
   'pursuer_4': np.float64(-104.50749893251731),
   'pursuer_5': np.float64(-180.21766134359572)},
  'policy_reward_max': {'pursuer_0': np.float64(-8.571273725909617),
   'pursuer_1': np.float64(60.30311215169101),
   'pursuer_2': np.float64(42.74708175708947),
   'pursuer_3': np.float64(-7.6108996017414645),
   'pursuer_4': np.float64(67.86120096904311),
   'pursuer_5': np.float64(-44.10802049107301)},
  'policy_reward_mean': {'pursuer_0': np.float64(-42.284014583047856),
   'pursuer_1': np.floa

Test for training:

In [4]:
resto_algo.train()

2024-09-03 17:55:20,672	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


{'evaluation': {'env_runners': {'episode_reward_max': np.float64(-364.769848341295),
   'episode_reward_min': np.float64(-619.0375683355406),
   'episode_reward_mean': np.float64(-493.78093420107905),
   'episode_len_mean': np.float64(500.0),
   'episode_media': {},
   'episodes_timesteps_total': 5000,
   'policy_reward_min': {'pursuer_0': np.float64(-113.90485384019159),
    'pursuer_1': np.float64(-109.27251741852379),
    'pursuer_2': np.float64(-114.55402342215696),
    'pursuer_3': np.float64(-111.76090622319133),
    'pursuer_4': np.float64(-109.0597747992956),
    'pursuer_5': np.float64(-217.50166684483233)},
   'policy_reward_max': {'pursuer_0': np.float64(11.303415129637894),
    'pursuer_1': np.float64(0.2069972847033158),
    'pursuer_2': np.float64(-46.84567547040106),
    'pursuer_3': np.float64(-6.933874989835202),
    'pursuer_4': np.float64(-7.903867561959164),
    'pursuer_5': np.float64(-102.74657319368887)},
   'policy_reward_mean': {'pursuer_0': np.float64(-62.6888

## Testing Callbacks



In [49]:
import ray
from ray.rllib.algorithms.callbacks import DefaultCallbacks

from typing import Dict, Tuple
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.policy import Policy

import numpy as np


class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, episode: Episode, **kwargs) -> None:
        episode.hist_data["mean_agent_return"] = []

    def on_episode_end(self, *, episode: Episode, **kwargs) -> None:
        episode.hist_data["mean_agent_return"].append(
            episode.total_reward / len(policies) )
        #episode.hist_data['mod_return'] = episode.hist_data['episode_reward']/2
    
    def on_train_result(self, *, algorithm, result: dict, **kwargs) -> None:
        #n_agents = len(policies)        
        n_agents = len(result['info']['learner'])
        result["num_agents"] = n_agents
        #result["mod_return"] = result['info']['hist_stats']["episode_reward"]/2
        result["mod_return"] = np.divide(
            result['env_runners']['hist_stats']['episode_reward'], n_agents)


ray.shutdown()


num_agents = 2
register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}

config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    #.evaluation(
    #    evaluation_interval=1,
    #)
    .callbacks(MyCallbacks)
    .training(trained_agents=num_agents)
)

algo = config.build()

TypeError: AlgorithmConfig.training() got an unexpected keyword argument 'trained_agents'

In [47]:
results = algo.train()

In [48]:
results

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0),
     'grad_gnorm': np.float32(2.6210372),
     'cur_kl_coeff': np.float64(0.20000000000000004),
     'cur_lr': np.float64(5.0000000000000016e-05),
     'total_loss': np.float64(8.687403419613839),
     'policy_loss': np.float64(-0.005285420302243438),
     'vf_loss': np.float64(8.691376147667567),
     'vf_explained_var': np.float64(-7.799391945203145e-06),
     'kl': np.float64(0.006563618332711485),
     'entropy': np.float64(2.8232320780555407),
     'entropy_coeff': np.float64(0.0)},
    'model': {},
    'custom_metrics': {},
    'num_agent_steps_trained': np.float64(125.0),
    'num_grad_updates_lifetime': np.float64(480.5),
    'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)},
   'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0),
     'grad_gnorm': np.float32(1.7212684),
     'cur_kl_coeff': np.float64(0.200000000

In [38]:
results['env_runners']['hist_stats']['episode_reward']
np.divide(results['env_runners']['hist_stats']['episode_reward'],2)


[np.float64(-224.56955954147745),
 np.float64(-210.60704272678404),
 np.float64(-270.4674606087424),
 np.float64(-300.55186835564245),
 np.float64(-269.2034552824137),
 np.float64(-225.00378578885667),
 np.float64(-221.94251529451898),
 np.float64(-224.5826295071694),
 np.float64(-225.01705161938244),
 np.float64(-289.68626249930134),
 np.float64(-244.16540430415856),
 np.float64(-266.06146701715795),
 np.float64(-177.1772171119893),
 np.float64(-185.088584697531),
 np.float64(-243.36592972517653),
 np.float64(-225.87668550926975)]

In [45]:
len(results['info']['learner'])

2